# Creating the Finite-Difference Table for Centered First and Second  Derivatives, from Second through Eighth-Order Accuracy

## Courtesy Brandon Clark

In [1]:
# Step 0: Import needed modules
import numpy as np
import finite_difference as fin
from astropy.table import Table

# Step 1: Set the maximum finite-difference accuracy order
max_fdorder = 8

# Step 2: Set up table parameters
#    One column for deriv order, one for deriv accuracy, and max_fdorder+1
numcols = 2 + max_fdorder + 1
#    8 rows: 4 accuracy orders per derivative order.
numrows = 8
#    Center column index of table will be at 2 + max_fdorder/2  (zero-offset indexing)
column_corresponding_to_zero_fd_point = 2 + int(max_fdorder/2)
#    The table is initialized as a matrix of zeroes in numpy:
numpy_matrix = np.zeros((numrows, numcols), dtype=object)

# Step 3: Construct the first-order derivative finite difference coefficients
rowcount = 0
for fdorder in range(2, max_fdorder+1, 2): # loop runs from 2 to max_fdorder inclusive, skipping odd orders.
    numpy_matrix[rowcount, 0] = "First"
    numpy_matrix[rowcount, 1] = fdorder     
    fdcoeffs, fdstencl = fin.compute_fdcoeffs_fdstencl("D0", fdorder)
    for i in range(fdorder):
        numpy_matrix[rowcount, column_corresponding_to_zero_fd_point + fdstencl[i][0]] = fdcoeffs[i]
    rowcount += 1

# Step 4: Construct the first-order derivative finite difference coefficients
for fdorder in range(2, max_fdorder+1, 2): # loop runs from 2 to max_fdorder inclusive, skipping odd orders.
    numpy_matrix[rowcount, 0] = "Second"
    numpy_matrix[rowcount, 1] = fdorder
    fdcoeffs, fdstencl = fin.compute_fdcoeffs_fdstencl("DD00", fdorder)
    for i in range(fdorder+1):
        numpy_matrix[rowcount, column_corresponding_to_zero_fd_point + fdstencl[i][0]] = fdcoeffs[i]
    rowcount += 1  

# Step 5: Construct an astropy table from the numpy matrix with the following header info, and then print it:
table = Table(numpy_matrix, names=('Derivative', 'Accuracy', '-4', '-3', '-2', '-1', '0', '1', '2', '3', '4'))
print(table)

Derivative Accuracy   -4     -3     -2   -1     0     1    2     3     4   
---------- -------- ------ ------ ----- ---- ------- --- ----- ----- ------
     First        2      0      0     0 -1/2       0 1/2     0     0      0
     First        4      0      0  1/12 -2/3       0 2/3 -1/12     0      0
     First        6      0  -1/60  3/20 -3/4       0 3/4 -3/20  1/60      0
     First        8  1/280 -4/105   1/5 -4/5       0 4/5  -1/5 4/105 -1/280
    Second        2      0      0     0    1      -2   1     0     0      0
    Second        4      0      0 -1/12  4/3    -5/2 4/3 -1/12     0      0
    Second        6      0   1/90 -3/20  3/2  -49/18 3/2 -3/20  1/90      0
    Second        8 -1/560  8/315  -1/5  8/5 -205/72 8/5  -1/5 8/315 -1/560
